In [57]:

# reading in the data via the Kaggle API

# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [58]:

# install Kaggle
! pip install kaggle

In [59]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [60]:
#read in your Kaggle credentials from Google Drive
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [61]:
# download the dataset from the competition page
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023
from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

detecting-french-texts-difficulty-level-2023.zip: Skipping, found more recently modified local copy (use --force to force download)


In [62]:
# read in your training data
import pandas as pd
import numpy as np

training = pd.read_csv('training_data.csv', index_col = 'id')

In [63]:
training.head()
training.dropna()
training.drop_duplicates()


,sentence,difficulty
id,,
0,Les coûts kilométriques réels peuvent diverger...,C1
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,Le test de niveau en français est sur le site ...,A1
3,Est-ce que ton mari est aussi de Boston?,A1
4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...
4795,"C'est pourquoi, il décida de remplacer les hab...",B2
4796,Il avait une de ces pâleurs splendides qui don...,C1
4797,"Et le premier samedi de chaque mois, venez ren...",A2


The first thing to do in order to train/test the data is to encode the column difficulty. We used labelencoder to have a new column with A1 = 0 ... C2 =5


In [64]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#OHE = OneHotEncoder()
#OHE.fit(training[['difficulty']])

#OHE_diff = OHE.transform(training[['difficulty']])
#OHE_diff_df = pd.DataFrame(OHE_diff.toarray(), columns=OHE.get_feature_names_out(['difficulty']))
#training_concat = pd.concat([training, OHE_diff_df], axis=1)
#training_concat = training_concat.drop('difficulty', axis=1)
label_encoder = LabelEncoder()
training['encoded_diff'] = label_encoder.fit_transform(training['difficulty'])


In [65]:
%%capture
!python -m spacy download fr_core_news_lg


Transform our sentences to spacy + tokenize

In [66]:
import numpy as np
import spacy
sp = spacy.load('fr_core_news_lg')
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS



In [67]:
#function that tokenize, takes out stopwords, and counts token in df
def tokenize_stop_words_count(df):
  df['sentence'] = df['sentence'].apply(sp)
  df['tokens'] = df['sentence'].apply(lambda doc: [token.text for token in doc])
  df['tokens_no_stop'] = df['tokens'].apply(lambda tokens: [token for token in tokens if token.lower() not in spacy_stopwords])
  df['token_count'] = df['tokens_no_stop'].apply(len)
  return df

In [68]:
training = tokenize_stop_words_count(training)

Try a LogisticRegression with y = the level and x = the nb of tokens

In [69]:
y = training['encoded_diff']
X = training[['token_count']]

Split into train/test

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


Reshape X_train because need 2D array

In [71]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


Check accuracy of predicted values

In [72]:
# Accuracy on the test set
print('Accuracy of Logistic regression classifier on test set: {:.2f}'
     .format(model.score(X_test, y_test)))

# Accuracy on the training set
print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(model.score(X_train, y_train)))

Accuracy of Logistic regression classifier on test set: 0.37
Accuracy of Logistic regression classifier on training set: 0.34


Not really good accuracy


In [73]:
from sklearn.metrics import accuracy_score
accuracy_test = accuracy_score(y_test, y_pred)

print(f'Accurary of Logistic regression classifier on test set: {accuracy_test :.2f}')

Accurary of Logistic regression classifier on test set: 0.37


In [74]:
test = pd.read_csv('unlabelled_test_data.csv')

In [75]:
test = tokenize_stop_words_count(test)

In [76]:
test.head()

,id,sentence,tokens,tokens_no_stop,token_count
0,0,"(Nous, dûmes, nous, excuser, des, propos, que,...","[Nous, dûmes, nous, excuser, des, propos, que,...","[dûmes, excuser, propos, eûmes, prononcés]",5
1,1,"(Vous, ne, pouvez, pas, savoir, le, plaisir, q...","[Vous, ne, pouvez, pas, savoir, le, plaisir, q...","[pouvez, savoir, plaisir, recevoir, bonne, nou...",7
2,2,"(Et, ,, paradoxalement, ,, boire, froid, n', e...","[Et, ,, paradoxalement, ,, boire, froid, n', e...","[,, paradoxalement, ,, boire, froid, bonne, pa...",8
3,3,"(Ce, n', est, pas, étonnant, ,, car, c', est, ...","[Ce, n', est, pas, étonnant, ,, car, c', est, ...","[étonnant, ,, saison, mystérieuse]",4
4,4,"(Le, corps, de, Golo, lui-même, ,, d', une, es...","[Le, corps, de, Golo, lui-même, ,, d', une, es...","[corps, Golo, ,, essence, surnaturelle, montur...",40


In [77]:
X_to_predict = test[['token_count']]

In [78]:
test['difficulty encoded'] = model.predict(X_to_predict)

In [95]:
#function that cleans and return the final dataset to upload on kaggle
def prep_final_pred(df):
  number_to_level = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

  df['difficulty'] = df['difficulty encoded'].map(number_to_level)
  columns_to_keep = ['id', 'difficulty']
  final_df =  df[columns_to_keep]
  final_df.set_index('id', inplace=True)


  return final_df


In [96]:
final_test = prep_final_pred(test)

In [97]:
final_test.head()

,difficulty
id,
0,A1
1,A2
2,B1
3,A1
4,C2


In [98]:
final_test.to_csv('submission.csv')


In [99]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

,id,difficulty
0,0,A1
1,1,A1
2,2,A1
3,3,A1
4,4,A1


In [100]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"


100% 8.30k/8.30k [00:00<00:00, 8.51kB/s]
Successfully submitted to Detecting the difficulty level of French texts

1)Compter le nb de token

> Bloc en retrait


2)Lemmatizer + créer une colonne qui compte cmb de mots ont été lemmatisé dans la phrase
3)